In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

def crawl_and_save_urls(base_url, output_file, exclude_url):
    """
    Crawl a website starting from the given base URL, extract links, and save them to a file.

    This function performs a breadth-first search (BFS)-like crawl of a website, starting from
    the base URL. It writes the visited URLs to an output file and avoids revisiting URLs.
    Additionally, it excludes URLs containing a specified substring.

    Args:
        base_url (str): The starting URL for the web crawler.
        output_file (str): The path to the file where the URLs will be saved.
        exclude_url (str): A substring to exclude URLs from crawling (e.g., external links).

    Behavior:
        - Fetches the HTML content of each page.
        - Extracts and normalizes all `<a>` tag links (using the `href` attribute).
        - Filters out already visited URLs or those containing the `exclude_url` substring.
        - Appends each visited URL to the specified output file.
        - Handles HTTP errors, parsing issues, and other exceptions gracefully.

    Raises:
        None: All exceptions are caught and logged for debugging purposes.

    Returns:
        None: The results are saved to the specified output file.
    """
    visited_urls = set()
    urls_to_visit = [base_url]
    
    # Open the file in append mode
    with open(output_file, 'a') as file:
        while urls_to_visit:
            url = urls_to_visit.pop(0)
            
            if url in visited_urls or exclude_url in url:
                continue
            
            try:
                response = requests.get(url)
                response.raise_for_status()  # Raise an error for bad responses
                soup = BeautifulSoup(response.text, 'html.parser')
            except requests.RequestException as e:
                print(f"Error fetching {url}: {e}")
                continue
            except Exception as e:
                print(f"Error parsing {url}: {e}")
                continue
            
            visited_urls.add(url)
            
            # Append the found URL to the file
            try:
                file.write(url + '\n')
                print(f"Added {url}")
            except Exception as e:
                print(f"Error writing {url} to file: {e}")
                continue
            
            # Find all anchor tags and extract their href attributes
            for link in soup.find_all('a', href=True):
                try:
                    href = link['href']
                    full_url = urllib.parse.urljoin(base_url, href)  # Construct full URL
                    if full_url.startswith(base_url) and full_url not in visited_urls and exclude_url not in full_url:
                        urls_to_visit.append(full_url)
                except Exception as e:
                    print(f"Error processing link {link}: {e}")
                    continue

if __name__ == "__main__":
    base_url = "https://www.stjohns.edu/"
    output_file = "urls.txt"
    exclude_url = "https://www.givecampus.com/"
    crawl_and_save_urls(base_url, output_file, exclude_url)
    print(f"Crawling completed. URLs saved to {output_file}")


In [ ]:
import os
import mimetypes
import requests
import nest_asyncio
import asyncio
from urllib.parse import urljoin
from playwright.async_api import async_playwright
from langchain.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader
from langchain.schema import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Apply the nest_asyncio patch
nest_asyncio.apply()

# Load your environment file
load_dotenv()

# Pinecone index configuration
index_name = os.environ.get('PINECONE_INDEX')
embeddings = OpenAIEmbeddings()
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)

# Helper functions

def get_file_type(url):
    """
    Determine the MIME (Multipurpose Internet Mail Extensions) type of a file based on its URL or filename.

    Args:
        url (str): The URL or filename to analyze.

    Returns:
        str: The MIME type of the file (e.g., 'application/pdf', 'text/html').
    """
    file_type, _ = mimetypes.guess_type(url)
    return file_type

def download_file(url, path):
    """
    Download a file from the given URL and save it to the specified path.

    Args:
        url (str): The URL of the file to download.
        path (str): The local path to save the downloaded file.

    Returns:
        str: The file path if download is successful.
        None: If the download fails.
    """
    try:
        response = requests.get(url, verify=False)
        response.raise_for_status()
        with open(path, "wb") as file:
            file.write(response.content)
        print(f"Downloaded: {url}")
        return path
    except Exception as e:
        print(f"Failed to download {url}: {e}")
        return None

async def fetch_page_content(url):
    """
    Fetch the HTML content of a webpage asynchronously.

    Args:
        url (str): The URL of the webpage to fetch.

    Returns:
        str: The HTML content of the webpage.
        None: If an error occurs during fetching.
    """
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        try:
            await page.goto(url)
            content = await page.content()
            await browser.close()
            return content
        except Exception as e:
            print(f"Failed to fetch {url}: {e}")
            await browser.close()
            return None

def clean_html(raw_html):
    """
    Clean raw HTML content by removing tags and extracting plain text.

    Args:
        raw_html (str): The raw HTML string to clean.

    Returns:
        str: The plain text extracted from the HTML.
    """
    soup = BeautifulSoup(raw_html, "html.parser")
    return soup.get_text(separator="\n", strip=True)

async def load_documents_async(url):
    """
    Load documents from a URL based on their file type.

    Args:
        url (str): The URL of the document or webpage.

    Returns:
        list: A list of Document objects with page content and metadata.
    """
    file_type = get_file_type(url)
    if file_type == "application/pdf":
        return PyPDFLoader(url).load()
    elif file_type in ["application/vnd.openxmlformats-officedocument.wordprocessingml.document", "application/msword"]:
        return UnstructuredWordDocumentLoader(url).load()
    else:
        content = await fetch_page_content(url)
        if content:
            clean_content = clean_html(content)
            metadata = {"source": url}  # Only include URL as metadata
            return [Document(page_content=clean_content, metadata=metadata)]
        else:
            return []

async def process_url_batch_async(url_batch):
    """
    Process a batch of URLs asynchronously, extracting documents and their linked files.

    Args:
        url_batch (list): A list of URLs to process.

    Returns:
        list: A list of all Document objects processed from the URLs.
    """
    all_documents = []
    for url in url_batch:
        try:
            documents = await load_documents_async(url)
            all_documents.extend(documents)

            # Process linked files in web pages
            for doc in documents:
                if "page_content" in doc.metadata:
                    soup = BeautifulSoup(doc.page_content, "html.parser")
                    for link in soup.find_all("a", href=True):
                        href = link["href"]
                        # Skip unwanted links
                        if "/email-protection#" in href:
                            print(f"Skipping link containing '/email-protection#': {href}")
                            continue
                        if href.endswith(".pdf") or href.endswith(".docx"):
                            file_url = href if href.startswith("http") else urljoin(url, href)
                            file_path = os.path.join("downloads", file_url.split("/")[-1])
                            if download_file(file_url, file_path):
                                if file_path.endswith(".pdf"):
                                    linked_docs = PyPDFLoader(file_path).load()
                                elif file_path.endswith(".docx"):
                                    linked_docs = UnstructuredWordDocumentLoader(file_path).load()
                                all_documents.extend(linked_docs)

        except Exception as e:
            print(f"Error processing {url}: {e}")
    return all_documents

def deduplicate_documents(docs):
    """
    Remove duplicate documents based on their page content.

    Args:
        docs (list): A list of Document objects.

    Returns:
        list: A list of unique Document objects.
    """
    seen = set()
    unique_docs = []
    for doc in docs:
        if doc.page_content not in seen:
            seen.add(doc.page_content)
            unique_docs.append(doc)
    return unique_docs

async def process_batches(urls, batch_size):
    """
    Process a large list of URLs in smaller batches asynchronously.

    Args:
        urls (list): The list of URLs to process.
        batch_size (int): The number of URLs to process per batch.

    Returns:
        None
    """
    for i in range(0, len(urls), batch_size):
        print(f"Processing batch {i // batch_size + 1}...")
        url_batch = urls[i:i + batch_size]

        # Process URL batch asynchronously
        all_documents = await process_url_batch_async(url_batch)

        # Deduplicate documents
        unique_docs = deduplicate_documents(all_documents)

        # Split documents into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
        docs = text_splitter.split_documents(unique_docs)

        # Upload chunks to Pinecone
        try:
            PineconeVectorStore.from_documents(
                docs, index_name=index_name, embedding=embeddings
            )
            print(f"Uploaded batch {i // batch_size + 1} to Pinecone.")
        except Exception as e:
            print(f"Error uploading batch {i // batch_size + 1}: {e}")

# Main script entry point
if __name__ == "__main__":
    # Load URLs from text file
    with open("urls.txt", "r") as f:
        urls = [line.strip() for line in f if line.strip()]

    # Process URLs in batches and upload directly to Pinecone
    asyncio.run(process_batches(urls, batch_size=100))



/opt/anaconda3/envs/LangChain_env/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Processing batch 1...
Uploaded batch 1 to Pinecone.
Processing batch 2...
Failed to fetch https://www.stjohns.edu/alumni-and-friends/get-involved/events-and-programs: Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://www.stjohns.edu/alumni-and-friends/get-involved/events-and-programs", waiting until "load"

Failed to fetch https://www.stjohns.edu/st-johns-university-alumni-friends/get-involved/virtual-book-remembrance: Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://www.stjohns.edu/st-johns-university-alumni-friends/get-involved/virtual-book-remembrance", waiting until "load"

Failed to fetch https://www.stjohns.edu/alumni-and-friends/get-involved/christmas-puzzle: Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://www.stjohns.edu/alumni-and-friends/get-involved/christmas-puzzle", waiting until "load"

Failed to fetch https://www.stjohns.edu/alumni-and-friends/alumni-benefits-and-resources/career-support: Page.goto: Timeout 3

Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)


Failed to fetch https://www.stjohns.edu/news-media/st-johns-media/associate-professor-criminal-justice-and-homeland-security-james-okeefe-phd-selecting-safe-place: Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://www.stjohns.edu/news-media/st-johns-media/associate-professor-criminal-justice-and-homeland-security-james-okeefe-phd-selecting-safe-place", waiting until "load"

Failed to fetch https://www.stjohns.edu/offices-departments/art-and-design: Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://www.stjohns.edu/offices-departments/art-and-design", waiting until "load"

Failed to fetch https://www.stjohns.edu/offices-departments?school=11: Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://www.stjohns.edu/offices-departments?school=11", waiting until "load"

Failed to fetch https://www.stjohns.edu/offices-departments/arts-and-humanities: Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://www.stjohns.edu/offi

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 28 0 (offset 0)
Ignoring wrong pointing object 33 0 (offset 0)
Ignoring wrong pointing object 44 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 58 0 (offset 0)


...

Failed to fetch https://www.stjohns.edu/academics/schools-and-colleges/st-johns-college-liberal-arts-and-sciences/environmental-studies: Page.goto: Timeout 30000ms exceeded.
Call log:
navigating to "https://www.stjohns.edu/academics/schools-and-colleges/st-johns-college-liberal-arts-and-sciences/environmental-studies", waiting until "load"

Failed to fetch https://www.stjohns.edu/academics/schools-and-colleges/st-johns-college-liberal-arts-and-sciences/theology-and-religious-studies: Page.goto: net::ERR_ABORTED at https://www.stjohns.edu/academics/schools-and-colleges/st-johns-college-liberal-arts-and-sciences/theology-and-religious-studies
Call log:
navigating to "https://www.stjohns.edu/academics/schools-and-colleges/st-johns-college-liberal-arts-and-sciences/theology-and-religious-studies", waiting until "load"

Failed to fetch https://www.stjohns.edu/academics/commencement/class-2020/virtual-recognition-ceremonies/gradslides: Page.goto: Timeout 30000ms exceeded.
Call log:
navigatin

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 31 0 (offset 0)
Ignoring wrong pointing object 39 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 18 0 (offset 0)
Ignoring wrong pointing object 20 0 (offset 0)
Ignoring wrong pointing object 22 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)


Error processing https://www.stjohns.edu/sites/default/files/2024-07/Fall%202024%20Accommodation%20Tip%20Sheet.docx: no such file or directory: 'https://www.stjohns.edu/sites/default/files/2024-07/Fall%202024%20Accommodation%20Tip%20Sheet.docx'
Error processing https://www.stjohns.edu/sites/default/files/2024-10/Bar%20Exam%20Accommodation%20Tip%20Sheet%202025-2026.docx: no such file or directory: 'https://www.stjohns.edu/sites/default/files/2024-10/Bar%20Exam%20Accommodation%20Tip%20Sheet%202025-2026.docx'
Uploaded batch 143 to Pinecone.
Processing batch 144...
Uploaded batch 144 to Pinecone.
Processing batch 145...
Uploaded batch 145 to Pinecone.
Processing batch 146...
Error processing https://www.stjohns.edu/sites/default/files/2024-08/JD%20Advising%20Guide%20updated%208.13.24%20-%20A%20%281%29.docx: no such file or directory: 'https://www.stjohns.edu/sites/default/files/2024-08/JD%20Advising%20Guide%20updated%208.13.24%20-%20A%20%281%29.docx'
Failed to fetch https://www.stjohns.edu

Ignoring wrong pointing object 7 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)
Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 26 0 (offset 0)
Ignoring wrong pointing object 34 0 (offset 0)
Ignoring wrong pointing object 36 0 (offset 0)
Ignoring wrong pointing object 38 0 (offset 0)
Ignoring wrong pointing object 42 0 (offset 0)
Ignoring wrong pointing object 46 0 (offset 0)
Ignoring wrong pointing object 49 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)
Ignoring wrong pointing object 62 0 (offset 0)
Ignoring wrong pointing object 66 0 (offset 0)


Failed to fetch https://www.stjohns.edu/files/soe-2020-commencement-program: Page.goto: net::ERR_ABORTED at https://www.stjohns.edu/files/soe-2020-commencement-program
Call log:
navigating to "https://www.stjohns.edu/files/soe-2020-commencement-program", waiting until "load"

Failed to fetch https://www.stjohns.edu/files/tcb-2020-virtual-commencement-program: Page.goto: net::ERR_ABORTED at https://www.stjohns.edu/files/tcb-2020-virtual-commencement-program
Call log:
navigating to "https://www.stjohns.edu/files/tcb-2020-virtual-commencement-program", waiting until "load"

Failed to fetch https://www.stjohns.edu/files/tcb-2020-commencement-program: Page.goto: net::ERR_ABORTED at https://www.stjohns.edu/files/tcb-2020-commencement-program
Call log:
navigating to "https://www.stjohns.edu/files/tcb-2020-commencement-program", waiting until "load"

Error processing https://www.stjohns.edu/sites/default/files/2024-07/24_Law%20School_Commencement_Awards_Honors.docx: no such file or directory: 